In [2]:
%set_env SPARK_HOME=/opt/spark
import os
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = """\
--master spark://10.62.42.243:7077 
--packages datastax:spark-cassandra-connector:2.3.1-s_2.11,anguenot:pyspark-cassandra:0.9.0
--driver-memory 53g
--executor-memory 22g
--conf spark.local.dir=/var/spark
--conf spark.network.timeout=420000
--conf spark.shuffle.registration.timeout=42000
pyspark-shell
"""

# --driver-memory 70g
# --executor-memory 25g
# --conf spark.executor.heartbeatInterval=42000
# --conf num-executors=4
# --conf spark.shuffle.service.enabled=true
# --conf spark.dynamicAllocation.executorIdleTimeout=42000
# --conf spark.dynamicAllocation.enabled=true
# --conf spark.dynamicAllocation.minExecutors=4

import pyspark
from pyspark import SparkContext, SparkConf
findspark.init()
sc = pyspark.SparkContext(appName="BestRetry")

env: SPARK_HOME=/opt/spark


In [3]:
sc._conf.set("spark.sql.execution.arrow.enabled", "true")
sc._conf.set('spark.driver.maxResultSize', '52')
sc._conf.set('spark.driver.cores', '4')
sc._conf.set('spark.executor.cores', '4')
# # sc._conf.set('spark.driver.memory', '24g')
# # sc._conf.set('spark.executor.memory', '24g')
sc._conf.set("spark.cores.max", 6)
sc._conf.set("spark.local.dir", "/var/spark")

# sc._conf.set("spark.network.timeout", "1000000s")
# sc._conf.set("spark.executor.heartbeatInterval", "1000000s")

# sc._conf.set("spark.cassandra.input.split.size_in_mb", 32)
# sc._conf.set("spark.cassandra.splitCount", 16)
# sc._conf.set("spark.cassandra.connection.host", "10.224.12.32")

sc._conf.set("spark.cassandra.connection.host", "10.62.1.118")
sc._conf.set("spark.cassandra.auth.username", "mlprw")
sc._conf.set("spark.cassandra.auth.password", "q4RgwD$wK7*z")

sc._conf.getAll()

[('spark.app.id', 'app-20190103030508-0018'),
 ('spark.driver.cores', '4'),
 ('spark.deploy.spreadOut', 'true'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.port', '30673'),
 ('spark.local.dir', '/var/spark'),
 ('spark.files',
  'file:///root/.ivy2/jars/datastax_spark-cassandra-connector-2.3.1-s_2.11.jar,file:///root/.ivy2/jars/anguenot_pyspark-cassandra-0.9.0.jar,file:///root/.ivy2/jars/com.twitter_jsr166e-1.1.0.jar,file:///root/.ivy2/jars/org.joda_joda-convert-1.2.jar,file:///root/.ivy2/jars/io.netty_netty-all-4.0.33.Final.jar,file:///root/.ivy2/jars/commons-beanutils_commons-beanutils-1.9.3.jar,file:///root/.ivy2/jars/joda-time_joda-time-2.3.jar,file:///root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar,file:///root/.ivy2/jars/commons-collections_commons-collections-3.2.2.jar,file:///root/.ivy2/jars/com.datastax.spark_spark-cassandra-connector_2.11-2.3.0.jar,file:///root/.ivy2/jars/net.razorvine_pyrolite-4.10.jar,file:///root/.ivy2/jars/net.razorvine_serpent

In [4]:
import pyspark
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.\
    master("spark://10.62.42.243:7077").\
    appName("BestRetry").\
    config(conf=sc._conf).\
    getOrCreate()
# spark

sqlContext = SQLContext(sc)
sqlContext

In [4]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



def merge_by_sub(group):
   
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    df_failed_txn = group[group['event_uuid'] != df_last_attempt['event_uuid']]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-12-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

pdf_all = toPandas(sdf_filtered, 32)  
# pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

(626869, 53)

In [9]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all[pdf_all['subscription_id'] == '12530960201']

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
0,2018-12-07,avgstore,12530960201,7881612123,2018-12-07 07:13:45,6,NON3DS,PT BANK DBS INDONESIA,ID,525644,MasterCard,Titanium MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capabil...,1219,,,7,friday,623f5b5d-1fa7-489a-b81d-608914b8bfbf,,Credit,ID,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,249000.0,17.1,IDR,MasterCard,netgiro-seb,4,249000.0,17.1,IDR,55157943200,27001,Transaction refused[51] [000000] [] [],New,17817501900,2018-12-07 01:13:45,01,Authorize,0,2018-12-07 01:13:48,7,302,,Base,0,5ff3c389-7da9-4070-9052-592f155754cd


In [5]:
pdf_all.to_csv(WORK_DIR + 'retry_success_12.csv')
# pdf_all.to_csv(WORK_DIR + 'retry_success_01_to_10.csv')

In [5]:
pdf_all['success'].value_counts(normalize=True)

0    0.845805
1    0.154195
Name: success, dtype: float64

In [10]:
sc.stop()
# pdf_all.to_csv(WORK_DIR + 'retry_success_02.csv')

In [8]:
#sdf_filtered.to_csv('best_retry_01_02.csv')
WORK_DIR = '/var/spark/ml_files/'
pdf_all.to_csv(WORK_DIR + 'best_retry_03.csv')
pdf_all.shape

(920431, 53)

In [7]:
pdf_all

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,...,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
0,2018-03-27,avast,4679748701,6606831463,2018-03-27 00:47:41,,F26EBCDBE58D0EC4,USAA Federal Savings Bank,US,456367,...,00,Authorize,0,2018-03-12 00:45:30,3,502,304: Lost/Stolen Card,Hard Fraud,15,01806eb8-b6ab-4089-9cc6-d6a4685a3448
1,2018-03-30,avastbr,4781058901,6617138493,2018-03-30 00:48:50,,NON3DS,Banco do Brasil S.A.,BR,498401,...,00,Authorize,1,2018-03-25 00:54:16,3,530,7: Do Not Honor,do_not_honor,5,ee93c0aa-34e2-4585-b88b-3a7e3893ac7c
2,2018-03-28,adbehkr,11195528901,6611214263,2018-03-28 02:41:27,,NON3DS,"WOORICARD CO., LTD",KR,538920,...,02,Authorize,0,2018-03-14 01:58:52,3,27050,Do not try again/use alternate payment card.[6...,Hard Fraud,14,19f3d80e-844b-46f3-a13d-a8f0a0583beb
3,2018-03-29,adbehbr,10798133101,6614426433,2018-03-29 01:06:29,,862D58E247602019,ITAU UNIBANCO S.A.,BR,548474,...,01,Authorize,1,2018-03-15 01:33:58,3,530,7: Do Not Honor,do_not_honor,14,a0ca5920-f1f4-4270-8fee-2045b609f67d
4,2018-03-03,avast,4010512701,6528156723,2018-03-03 02:30:17,4,RB,La Banque Postale,FR,415059,...,02,Authorize,0,2018-03-03 02:30:19,3,27001,Transaction refused[05] [] [111800] [111800],txn_refused,0,d18b7672-4764-4ecb-bb18-7cc416b593bf
5,2018-03-30,avast,5343854301,6617090023,2018-03-30 00:43:15,3,NON3DS,BBVA BANCO FRANCES,AR,529329,...,00,Authorize,0,2018-03-30 00:43:16,3,27001,Transaction refused[05] [] [111800] [111800],txn_refused,0,54f62a99-7b88-4916-8c7a-c5a41102ef2f
6,2018-03-26,adbehap,9262717601,6604096753,2018-03-26 01:24:43,,NON3DS,SBI Cards and Payment Services Pte Ltd,IN,472642,...,01,Authorize,0,2018-03-05 01:36:21,3,27001,Transaction refused[05] [000000] [] [],txn_refused,21,a529594e-690a-43c8-9b6f-8e65a233c659
7,2018-03-15,adbehru,11121914701,6568069593,2018-03-15 02:31:39,,NON3DS,TINKOFF BANK,RU,553691,...,02,Authorize,1,2018-03-08 01:44:03,3,27001,Transaction refused[51] [000000] [] [],Insufficient Funds,7,70fd7b9b-5de8-4089-836e-493b5a38581d
8,2018-03-24,avast,9906987801,6597665833,2018-03-24 01:28:48,,E6B9CECFCAC4D317,JPMorgan Chase Bank N.A. - Debit,US,483314,...,01,Authorize,0,2018-03-09 01:26:40,3,530,51: DECLINED,declined,15,ce2b12b9-941d-45a8-8d58-495ba5ed57b0
9,2018-03-24,kasperus,9885148101,6597466793,2018-03-24 01:01:06,,D0ECAF1CD662089E,Equitable Bank,US,413976,...,01,Authorize,0,2018-03-07 09:05:32,3,530,51: DECLINED,declined,17,f4ddead6-68b9-45cd-9155-d5bf5528dea1


In [12]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'
# start_date = '2018-04-01'
# end_date = '2018-06-30'

def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
        date_formatted = date.replace('-','')
        yield date_formatted

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    
    df_failed_txn = group[group['event_uuid'] != df_last_attempt['event_uuid']]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

# @pandas_udf(sdf.schema, PandasUDFType.GROUPED_MAP)

        

# date_list = list(date_range_condition_values(start_date, end_date))


udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed','Reversed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                    
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

year=2018
months = ['04','05','06']
for m in months:
    yearmonth = str('%s-%s'%(year,m))
    condition = str("received_date like '%s%%'" % yearmonth)
    filename = str("retry_success_%s.csv" % yearmonth)
    sdf_by_month = sdf\
        .where(condition)\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")
    
#     sdf_by_month.toPandas().to_csv(WORK_DIR + filename) 
    toPandas(sdf_by_month, 4).to_csv(WORK_DIR + filename) 
    
    
print("# total time:", time.time() - time_start)    
    
# pdf_all

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

ValueError: Length mismatch: Expected axis has 0 elements, new values have 53 elements

In [13]:
sc.stop()

In [4]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

# rdd = sc \
#     .cassandraTable("dev_sarsks", "cpg_transaction_history")\
#     .select('site_id')

# #     .map(lambda a:a)

# #     .where("received_date in ?", date_list)

# sdf = spark.read\
#     .format("org.apache.spark.sql.cassandra")\
#     .options(table="cpg_transaction_history", keyspace="dev_sarsks")\
#     .load() 

# df = sqlContext.createDataFrame(rdd, StructType([StructField('site_id',StringType(),True)]))

# df = sqlContext.read.format('org.apache.spark.sql.cassandra').options(table="cpg_transaction_history", keyspace="dev_sarsks")\
#     .where("received_date = ?", '2018-01-01')\
#     .load()
# df.select('id','date').where(df.date=='2017-12-01').show()
# df.where( df.received_date == '2018-01-01').head()
# df = sqlContext.createDataFrame(rdd, StructType([StructField('site_id',StringType(),True),StructField('payment_currency',StringType(),True)]))

# sdf = df.withColumn("received_date", df["received_date"].cast(StringType()))
# df = sqlContext.createDataFrame(rdd, sdf.schema)
# df.head()

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "Test Cluster",
     pushdown "true")"""
spark.sql(createDDL)

spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-02-%'").show()
# df.head()
# print(rdd.count)      

+-------------+-------+---------------+--------------+-------------------+------------------+----------------+--------------------+---------------+------+----------------+--------------------+----------+--------------------+------------------+----------------+--------------+------------+-----------+--------------------+--------------+--------------+--------------+-------------------+--------------------+-----------------+----------+-----------------+--------------+------------------+----------------+-----------------+------------------+-------------+--------------+------------------+----------------+--------------+-------------+--------------------+------+-------------+--------------------------+----------------+----------------+
|received_date|site_id|subscription_id|transaction_id|   transaction_date|added_expiry_years|       bank_code|           bank_name|billing_country|   bin|      card_brand|       card_category|card_class|          card_usage|cc_expiration_date|             cid|

In [9]:
# pdf_all['day_of_week'].value_counts(dropna=False)
# pdf_all.to_csv('/drhdev/workspace/ml/pdf_all_site_2018-01-01_2018-06-30.csv')
from IPython.display import display
pd.options.display.max_columns = None
display(pdf_all)

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,first_attempt_date,first_day_of_month,first_transaction_hour,first_response_code,first_response_message,first_response_group,first_decline_type,days_between
0,20180418,avast,10001280501,673671cc-cc57-400c-b872-8ff9efb09a6d,None,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,0322,None,None,18,wednesday,None,Debit,TZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,79.99,USD,Visa,netgiro-bms,3,79.99,79.99,USD,46971082700,27050,Do not try again/use alternate payment card.[0...,None,New,13258980000,,2018-04-18 06:21:29,2018-04-18 01:21:29,01,6681738323,Authorize,0,2018-04-03 01:07:59,3,01,27050,Do not try again/use alternate payment card.[0...,do_not_try,Hard Fraud,15
1,20180418,avastbr,10001441601,11cc8ab0-fe20-4e80-9d8c-2a7a856e7f17,None,NON3DS,ITAU UNIBANCO S.A.,BR,539083,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),1223,None,None,18,wednesday,None,Credit,BR,Avast Brazil,1883448942,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,99,29.018051,BRL,MasterCard,netgiro-br,3,99,29.018051,BRL,46969296500,27001,Transaction refused[62] [null] [] [],None,New,13259159000,,2018-04-18 06:02:11,2018-04-18 01:02:11,01,6681601083,Authorize,0,2018-04-08 01:03:36,8,01,27001,Transaction refused[62] [null] [] [],txn_refused,Hard Fraud,10
2,20180418,avast,10001635101,8bc95f07-77b3-4bd4-abfa-5fee6658eb63,None,57E041B0B2F3D9A5,Metabank,US,435880,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0625,None,None,18,wednesday,None,Prepaid,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,53.99,53.99,USD,Visa,firstdata,3,53.99,53.99,USD,46969386300,402,12: INV TRAN,None,New,13259490100,,2018-04-18 06:03:02,2018-04-18 01:03:02,01,6681607843,Authorize,0,2018-04-03 01:10:44,3,01,402,12: INV TRAN,Base,Base,15
3,20180425,defendbr,10001895901,9f5ecc0d-2529-4930-9fc5-93daf88c9e74,3,899EBF821D876833,ITAU UNIBANCO S.A.,BR,521397,MasterCard,Platinum MasterCard Card,Consumer,True credit (No PIN/Signature capability),0620,None,None,25,wednesday,None,Credit,BR,DRI*BitDefender,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,229.99,66.742732,BRL,MasterCard,pagador,3,229.99,66.742732,BRL,47202743400,530,7: Do Not Honor,None,New,13259627500,,2018-04-25 06:01:35,2018-04-25 01:01:35,01,6706709733,Authorize,0,2018-04-16 01:01:42,16,01,530,7: Do Not Honor,do_not_honor,do_not_honor,9
4,20180418,avast,10001966301,1bf185fb-88cd-457c-85a2-a697021fc538,None,0FDA420D7B7FDBDC,"Chase Bank USA, National Association",US,424631,Visa,Visa Business,Business,True credit (No PIN/Signature capability),0718,None,None,18,wednesday,None,Credit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.94,64.94,USD,Visa,firstdata,3,64.94,64.94,USD,46969539100,201,14: INV ACCT NUM,None,New,13259788300,,2018-04-18 06:05:07,2018-04-18 01:05:07,01,6681621783,Authorize,0,2018-04-03 01:12:21,3,01,201,14: INV ACCT NUM,Base,Base,15
5,20180519,adbehru,10002261401,0d1fbb95-019b-4f5f-8c53-f96ae78c2d0a,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1018,RETRY_DECLINED.2,None,19,saturday,None,Debit,RU,DRI*Adobe Systems,1411163460-R

In [18]:
'''Queries all successful transactions to generate bin profile'''
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date


    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-02-%' ")
df = spark.sql("SELECT bin, payment_amount_usd, subscription_id, received_date FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' ")

sdf =  df\
    .filter("new_status in ('Completed')")\
    .cache()

print(sdf.schema)
                                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

# sdf_filtered = sdf
#         .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf_filtered, 6)  
successful_txn = sdf.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
successful_txn.shape

StructType(List(StructField(bin,StringType,true),StructField(payment_amount_usd,StringType,true),StructField(subscription_id,StringType,true),StructField(received_date,DateType,true)))
# total time: 149.83152294158936


(11653576, 4)

In [22]:
successful_txn['payment_amount_usd'] = successful_txn['payment_amount_usd'].astype(float)
successful_txn['payment_amount_usd'] 

0             21.600000
1             17.050000
2             17.050000
3             85.080000
4             21.500000
5             17.050000
6             17.050000
7             83.740000
8             79.000000
9             20.520000
10            32.100000
11            32.520000
12            14.500000
13            14.900000
14            54.200000
15            49.600000
16            29.800000
17            10.700000
18            15.930000
19            14.700000
20            29.100000
21             2.210000
22            21.680000
23            31.800000
24            22.200000
25            21.400000
26            11.100000
27            18.800000
28             9.800000
29            26.500000
               ...     
11653546      18.638761
11653547      89.170000
11653548      44.950000
11653549      80.950000
11653550      48.100000
11653551      68.840000
11653552      86.820000
11653553      68.540000
11653554      80.950000
11653555      62.950000
11653556      66

In [24]:
bin_profile[bin_profile['bin']=='']

,bin,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,count
0,,10.231983,0.011639,410.0,19.355269,57.48,85.59,4.590894,3026


In [23]:
grouped = successful_txn['payment_amount_usd'].groupby(successful_txn['bin']).agg([('Mean','mean'),('Min','min'),('Max','max'),('StdDev','std'), ('Max_95', lambda x : x.quantile(.95)), ('Max_99', lambda x : x.quantile(.99)), ('Min_1', lambda x : x.quantile(.1)) , 'count']).reset_index()

bin_profile = grouped[grouped['count'] >= 5]
# bin_profile
bin_profile.to_csv(WORK_DIR + 'bin_profile.csv')
bin_profile

,bin,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,count
0,,10.231983,0.011639,410.000000,19.355269,57.480000,85.590000,4.590894,3026
5,352818,24.845652,4.839536,51.479167,19.995024,47.495588,50.682451,4.840481,5
7,352821,49.015144,48.102689,50.518630,0.988569,50.246824,50.464269,48.115695,5
8,352822,33.641090,18.714051,49.360468,10.450449,46.485601,48.785495,18.714051,9
12,352831,39.761784,18.336803,68.206773,17.110155,62.866110,67.138640,18.336803,8
13,352832,39.497431,31.606969,54.133242,9.201100,51.345292,53.575652,31.625561,5
17,352852,45.536429,30.898007,74.471954,16.389221,69.740368,73.525637,31.331695,7
19,352855,88.984131,18.420649,428.518055,149.894144,312.078075,405.230059,26.037511,7
20,352856,68.905002,8.685324,424.582865,105.066760,216.143403,382.894973,11.838375,14
21,352859,69.043849,7.813289,563.086969,112.394027,225.503786,559.748866,8.164704,82


In [25]:
bin_profile.shape

(30600, 9)

In [6]:
# pdf_all.to_csv(WORK_DIR + 'successful_txn_01_to_10.csv')

In [4]:
'''Queries all raw  transactions data'''
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date


    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-07-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' ")

sdf =  df\
    .cache()

print(sdf.schema)
    

                                             
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

# sdf_filtered = sdf
#         .filter("event_uuid != ''")  #.toPandas()

pdf_all_07 = toPandas(sdf, 64)  
# successful_txn = sdf.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all_07.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,StringType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant_

(2487901, 45)

In [7]:
# pdf_all_07.to_csv(WORK_DIR + 'raw_txns_07.csv')
pdf_all_07 = pd.read_csv(WORK_DIR + 'raw_txns_07.csv')

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [101]:
pdf_all_07[pdf_all_07['renew_att_num'] == '3'].head(5)['subsegment_id']

13    13865645800
14    16460118700
26    13791788100
47     5435866700
55    11815740300
Name: subsegment_id, dtype: int64

In [218]:
pdf_all_07[pdf_all_07['subsegment_id'] == '13865645800']

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
13,2018-07-25,avgstore,10363932201,7109736923,2018-07-25 06:28:30,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,25,Wednesday,7159bc5a-7e30-4ce3-9183-957c7c93c66e,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.258375,DKK,Visa,netgiro-bms,3,549,86.258375,DKK,50252786700,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-25 01:28:30,01,Authorize,0.0
29368,2018-07-10,avgstore,10363932201,7034206383,2018-07-10 06:44:58,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,10,Tuesday,f25a2734-918a-4725-933f-031a358efa2f,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.835015,DKK,Visa,netgiro-seb,1,549,86.835015,DKK,49782722000,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-10 01:44:58,01,Authorize,0.0
27718,2018-07-18,avgstore,10363932201,7074667233,2018-07-18 06:36:50,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,18,Wednesday,3f45c21c-df43-4dd2-8068-a045f1283cff,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.224081,DKK,Visa,netgiro-seb,2,549,86.224081,DKK,50032013300,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-18 01:36:50,01,Authorize,0.0
28528,2018-07-10,avgstore,10363932201,7034206523,2018-07-10 06:44:59,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,10,Tuesday,7a9e2bdc-3828-48cf-b956-5869d250fa72,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.835015,DKK,Visa,netgiro-bms,1,549,86.835015,DKK,49782722000,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-10 01:44:59,01,Authorize,0.0
20439,2018-07-18,avgstore,10363932201,7074667383,2018-07-18 06:36:52,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,18,Wednesday,e210e36f-737c-40a1-ae78-0bffad118468,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.224081,DKK,Visa,netgiro-bms,2,549,86.224081,DKK,50032013300,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-18 01:36:52,01,Authorize,0.0
7675,2018-07-25,avgstore,10363932201,7109736833,2018-07-25 06:28:29,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,25,Wednesday,db145677-1fa3-4b42-9893-0b65f0db58e4,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.258375,DKK,Visa,netgiro-seb,3,549,86.258375,DKK,50252786700,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-25 01:28:29,01,Authorize,0.0


In [222]:
pdf_all_07[pdf_all_07['subsegment_id'] == '13865645800'].sort_values(by=['renew_att_num'])[['renew_att_num', 'event_uuid']]
# pdf_all_07[pdf_all_07['subscription_id'] == '9171379701']

,renew_att_num,event_uuid
29368,1,f25a2734-918a-4725-933f-031a358efa2f
28528,1,7a9e2bdc-3828-48cf-b956-5869d250fa72
27718,2,3f45c21c-df43-4dd2-8068-a045f1283cff
20439,2,e210e36f-737c-40a1-ae78-0bffad118468
13,3,7159bc5a-7e30-4ce3-9183-957c7c93c66e
7675,3,db145677-1fa3-4b42-9893-0b65f0db58e4


In [103]:
pdf_all_07[pdf_all_07['subsegment_id'].isin([13865645800, 16460118700, 11815740300])].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_sub)[['event_uuid', 'failed_event_uuid', 'renew_att_num', 'received_date', 'success', 'failed_attempt_date', 'failed_response_code', 'failed_response_message', 'failed_decline_type', 'days_between']]
# pdf_all_07[pdf_all_07['subscription_id'] == '9171379701'].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_sub)

event_uuid  \
subscription_id subsegment_id                                                 
9140422001      11815740300   1013013  90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              2116761  9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              1013013  90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              2116761  9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              55       e91f63d9-e324-4259-bb11-2005b014ce81   
                              1866303  f4665fb1-bb21-4c6e-a2d0-1f72f1323445   
                              55       e91f63d9-e324-4259-bb11-2005b014ce81   
                              1866303  f4665fb1-bb21-4c6e-a2d0-1f72f1323445   
10363932201     13865645800   534786   90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              1886177  9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              534786   90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              1886177  9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              13       e91f63d9-e324-4259-bb11-2005b014ce81   
                              1466781  f4665fb1-bb21-4c6e-a2d0-1f72f1323445   
                              13       e91f63d9-e324-4259-bb11-2005b014ce81   
                              1466781  f4665fb1-bb21-4c6e-a2d0-1f72f1323445   
11831972601     16460118700   671473   90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              1828997  9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              671473   90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              1828997  9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              14       e91f63d9-e324-4259-bb11-2005b014ce81   
                              1222562  f4665fb1-bb21-4c6e-a2d0-1f72f1323445   
                              14       e91f63d9-e324-4259-bb11-2005b014ce81   
                              1222562  f4665fb1-bb21-4c6e-a2d0-1f72f1323445   

                                                          failed_event_uuid  \
subscription_id subsegment_id                                                 
9140422001      11815740300   1013013  659cc633-4ebf-4724-9854-7e4801bae033   
                              2116761  659cc633-4ebf-4724-9854-7e4801bae033   
                              1013013  6c27145d-0639-4f32-9542-dacf730b694f   
                              2116761  6c27145d-0639-4f32-9542-dacf730b694f   
                              55       90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              1866303  90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              55       9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              1866303  9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
10363932201     13865645800   534786   659cc633-4ebf-4724-9854-7e4801bae033   
                              1886177  659cc633-4ebf-4724-9854-7e4801bae033   
                              534786   6c27145d-0639-4f32-9542-dacf730b694f   
                              1886177  6c27145d-0639-4f32-9542-dacf730b694f   
                              13       90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              1466781  90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              13       9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              1466781  9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
11831972601     16460118700   671473   659cc633-4ebf-4724-9854-7e4801bae033   
                              1828997  659cc633-4ebf-4724-9854-7e4801bae033   
                              671473   6c27145d-0639-4f32-9542-dacf730b694f   
                              1828997  6c27145d-0639-4f32-9542-dacf730b694f   
                              14       90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              1222562  90c1f3e1-200c-4f08-8fd6-72601cd103b5   
                              14       9a9f116c-ad17-48f6-9ef5-dd43caf24b73   
                              12225

In [91]:
import itertools

# Merge failed attempt with the next attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    new_group = pd.DataFrame(columns = group.columns)
    
    sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]
    if not last_att_num or not initial_att_num:
        group['event_uuid'] = '' 
        return group.iloc[0]
        
    for att_num in range(int(initial_att_num), int(last_att_num)):
        next_att_num = int(att_num) + 1
        initial_att_indexes = group[group['renew_att_num'] == str(att_num)].index.tolist()
        next_att_indexes = group[group['renew_att_num'] == str(next_att_num)].index.tolist()
        
        for item in itertools.product(*[initial_att_indexes, next_att_indexes]):
            merged = group.loc[item[1]].copy(deep=True)
            inital_row = group.loc[item[0]]                   
            failed_attempt_date = inital_row['transaction_date_in_string']                   
            merged['failed_attempt_date'] = failed_attempt_date

            merged['failed_day_of_month'] = inital_row['day_of_month']
            merged['day_of_month'] = int(merged['day_of_month'])
            merged['failed_response_code'] = inital_row['response_code']
            
            failed_response_msg = inital_row['response_message']
            merged['failed_response_message'] = failed_response_msg  

            merged['failed_decline_type'] = decline_type(inital_row['response_message']) 
            merged['days_between'] = days_between_ds(merged)
            merged['failed_event_uuid'] =   inital_row['event_uuid']   
            new_group = new_group.append(merged)
            
           
    if new_group.empty:
        group['event_uuid'] = '' 
        return group.iloc[0]
    
    return new_group

In [25]:
# Merge only the last two renew_att_num with the last single attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    last_att_num = int(df_last_attempt['renew_att_num'])
    previous_att_num = last_att_num - 1
   
    df_failed_txn = group[(group['renew_att_num'].isin([ str(previous_att_num),  df_last_attempt['renew_att_num'] ])) & (group['event_uuid'] != df_last_attempt['event_uuid']) ]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last

In [6]:
# Merge failed attempt with the next attempt
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta
import itertools

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



# Merge failed attempt with the next attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    new_group = pd.DataFrame(columns = group.columns)
    
    sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]
    if not last_att_num or not initial_att_num:
        group['event_uuid'] = '' 
        return group
#         return group.iloc[0]
        
    for att_num in range(int(initial_att_num), int(last_att_num)):
        next_att_num = int(att_num) + 1
        initial_att_indexes = group[group['renew_att_num'] == str(att_num)].index.tolist()
        next_att_indexes = group[group['renew_att_num'] == str(next_att_num)].index.tolist()
        
        for item in itertools.product(*[initial_att_indexes, next_att_indexes]):
            merged = group.loc[item[1]].copy(deep=True)
            inital_row = group.loc[item[0]]                   
            failed_attempt_date = inital_row['transaction_date_in_string']                   
            merged['failed_attempt_date'] = failed_attempt_date

            merged['failed_day_of_month'] = inital_row['day_of_month']
            merged['day_of_month'] = int(merged['day_of_month'])
            merged['failed_response_code'] = inital_row['response_code']
            
            failed_response_msg = inital_row['response_message']
            merged['failed_response_message'] = failed_response_msg  

            merged['failed_decline_type'] = decline_type(inital_row['response_message']) 
            merged['days_between'] = days_between_ds(merged)
            merged['failed_event_uuid'] =   inital_row['event_uuid']   
            new_group = new_group.append(merged)
            
           
    if new_group.empty:
        group['event_uuid'] = '' 
        return group
#         return group.iloc[0]
    
    return new_group


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    print('d1: {}, d2: {}'.format(d1, d2))
    d1 = to_date(d1)
    d2 = to_date(d2)
    print('after days_between')
    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  
# udfValueToStatus = udf(convert_status, StringType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-12-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf_filtered, 128)  
pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

(1673882, 53)

In [7]:
pdf_all[pdf_all['success'] == 1]['renew_att_num'].value_counts(normalize=True)

2     0.604973
3     0.233627
4     0.098280
5     0.044809
6     0.017083
7     0.001133
8     0.000042
9     0.000042
10    0.000011
Name: renew_att_num, dtype: float64

In [8]:
# pdf_all.to_csv(WORK_DIR + 'retry_success_next_attempt_1_to_10.csv')
pdf_all.to_csv(WORK_DIR + 'retry_success_next_attempt_12.csv')

In [109]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all.head()

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
0,2018-12-10,avast,10019477901,7898877723,2018-12-10 15:45:46,,NON3DS,Iccrea Banca S.p.A. - Istituto Centrale del Cr...,IT,486470,Visa,Corporate T and E,Corporate,Credit - True credit (No PIN/Signature capabil...,0719,,,10,monday,0355f182-0fee-44f3-99f6-27dfdf961bea,,Deferred Debit,IT,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,40.98,46.60,EUR,Visa,netgiro-bms,3,40.98,46.60,EUR,55300966800,27001,Transaction refused[05] [] [111800] [111800],New,13296142400,2018-12-10 09:45:46,09,Authorize,0,2018-12-05 07:45:47,5,27001,Transaction refused[null] [null] [null] [null],txn_refused,5,a439cb8c-c234-488b-8a4a-f96a65cc7d19
1,2018-12-10,avast,10019477901,7898878403,2018-12-10 15:45:47,,NON3DS,Iccrea Banca S.p.A. - Istituto Centrale del Cr...,IT,486470,Visa,Corporate T and E,Corporate,Credit - True credit (No PIN/Signature capabil...,0719,RETRY_DECLINED.2,,10,monday,b17b54ca-6935-423a-9a66-b3c3221466fd,,Deferred Debit,IT,DRI*AVAST Software,1009072982,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,40.98,46.60,EUR,Visa,drwp-fd,3,40.98,46.60,EUR,55300966800,27001,Transaction refused[null] [null] [null] [null],New,13296142400,2018-12-10 09:45:47,09,Authorize,0,2018-12-05 07:45:47,5,27001,Transaction refused[null] [null] [null] [null],txn_refused,5,a439cb8c-c234-488b-8a4a-f96a65cc7d19
2,2018-12-10,avast,10019477901,7898877723,2018-12-10 15:45:46,,NON3DS,Iccrea Banca S.p.A. - Istituto Centrale del Cr...,IT,486470,Visa,Corporate T and E,Corporate,Credit - True credit (No PIN/Signature capabil...,0719,,,10,monday,0355f182-0fee-44f3-99f6-27dfdf961bea,,Deferred Debit,IT,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,40.98,46.60,EUR,Visa,netgiro-bms,3,40.98,46.60,EUR,55300966800,27001,Transaction refused[05] [] [111800] [111800],New,13296142400,2018-12-10 09:45:46,09,Authorize,0,2018-12-05 07:45:48,5,27001,Transaction refused[05] [] [111800] [111800],txn_refused,5,c98e3a1c-8cae-406a-8cb6-47d108babf94
3,2018-12-10,avast,10019477901,7898878403,2018-12-10 15:45:47,,NON3DS,Iccrea Banca S.p.A. - Istituto Centrale del Cr...,IT,486470,Visa,Corporate T and E,Corporate,Credit - True credit (No PIN/Signature capabil...,0719,RETRY_DECLINED.2,,10,monday,b17b54ca-6935-423a-9a66-b3c3221466fd,,Deferred Debit,IT,DRI*AVAST Software,1009072982,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,40.98,46.60,EUR,Visa,drwp-fd,3,40.98,46.60,EUR,55300966800,27001,Transaction refused[null] [null] [null] [null],New,13296142400,2018-12-10 09:45:47,09,Authorize,0,2018-12-05 07:45:48,5,27001,Transaction refused[05] [] [111800] [111800],txn_refused,5,c98e3a1c-8cae-406a-8cb6-47d108babf94
4,2018-12-12,adbehap,10051777801,7911511643,2018-12-12 06:37:33,,NON3DS,BOC Credit Card (International) Limited,HK,434662,Visa,"No Data Found', value: 'N/A",Business,Credit - True credit (No PIN/Signature capabil...,0521,,,12,wednesday,29caa62e-2111-4922-bd29-7fda12090546,,Credit,HK,DRI*Adobe Sales,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,108.0,13.8,HKD,Visa,netgiro-bms,2,108.0,13.8,HKD,55346067900,0,SUCCESS[00] [096863] [111800] [111800],New,17803489900,2018-12-12 00:37:33,00,Authorize,1,2018-12-05 00:36:37,5,27001,Transaction refused[05] [] [

In [9]:
# Merge only the last two renew_att_num with the last single attempt
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



# Merge only the last two renew_att_num with the last single attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    if df_last_attempt['renew_att_num']:
        last_att_num = int(df_last_attempt['renew_att_num'])
        previous_att_num = last_att_num - 1
    else:
        group['event_uuid'] = '' 
        return group
   
    df_failed_txn = group[(group['renew_att_num'].isin([ str(previous_att_num),  df_last_attempt['renew_att_num'] ])) & (group['event_uuid'] != df_last_attempt['event_uuid']) ]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-12-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

pdf_all = toPandas(sdf_filtered, 64)  
# pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

(824803, 53)

In [10]:
# pdf_all.to_csv(WORK_DIR + 'retry_success_two_last_att_num_01_to_10.csv')
pdf_all.to_csv(WORK_DIR + 'retry_success_two_12.csv')

In [85]:
pdf_all['success'].value_counts(normalize=True)

0    0.804565
1    0.195435
Name: success, dtype: float64

In [77]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all_07[pdf_all_07['subscription_id'] == '10363932201'].sort_values(by=['renew_att_num'])

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
29368,2018-07-10,avgstore,10363932201,7034206383,2018-07-10 06:44:58,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,10,Tuesday,f25a2734-918a-4725-933f-031a358efa2f,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.835015,DKK,Visa,netgiro-seb,1,549,86.835015,DKK,49782722000,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-10 01:44:58,01,Authorize,0.0
28528,2018-07-10,avgstore,10363932201,7034206523,2018-07-10 06:44:59,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,10,Tuesday,7a9e2bdc-3828-48cf-b956-5869d250fa72,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.835015,DKK,Visa,netgiro-bms,1,549,86.835015,DKK,49782722000,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-10 01:44:59,01,Authorize,0.0
27718,2018-07-18,avgstore,10363932201,7074667233,2018-07-18 06:36:50,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,18,Wednesday,3f45c21c-df43-4dd2-8068-a045f1283cff,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.224081,DKK,Visa,netgiro-seb,2,549,86.224081,DKK,50032013300,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-18 01:36:50,01,Authorize,0.0
20439,2018-07-18,avgstore,10363932201,7074667383,2018-07-18 06:36:52,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,18,Wednesday,e210e36f-737c-40a1-ae78-0bffad118468,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.224081,DKK,Visa,netgiro-bms,2,549,86.224081,DKK,50032013300,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-18 01:36:52,01,Authorize,0.0
13,2018-07-25,avgstore,10363932201,7109736923,2018-07-25 06:28:30,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,25,Wednesday,7159bc5a-7e30-4ce3-9183-957c7c93c66e,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.258375,DKK,Visa,netgiro-bms,3,549,86.258375,DKK,50252786700,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-25 01:28:30,01,Authorize,0.0
7675,2018-07-25,avgstore,10363932201,7109736833,2018-07-25 06:28:29,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,25,Wednesday,db145677-1fa3-4b42-9893-0b65f0db58e4,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.258375,DKK,Visa,netgiro-seb,3,549,86.258375,DKK,50252786700,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-25 01:28:29,01,Authorize,0.0


In [78]:
pdf_all_07[pdf_all_07['subscription_id'] == '10363932201'].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_sub)
# pdf_all_07[pdf_all_07['subscription_id'] == '11623213001'].groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)

received_date   site_id subscription_id  \
subscription_id subsegment_id                                                 
10363932201     13865645800   27718    2018-07-25  avgstore     10363932201   
                              20439    2018-07-25  avgstore     10363932201   
                              7675     2018-07-25  avgstore     10363932201   

                                    transaction_id    transaction_date  \
subscription_id subsegment_id                                            
10363932201     13865645800   27718     7109736923 2018-07-25 06:28:30   
                              20439     7109736923 2018-07-25 06:28:30   
                              7675      7109736923 2018-07-25 06:28:30   

                                    added_expiry_years bank_code  \
subscription_id subsegment_id                                      
10363932201     13865645800   27718                       NON3DS   
                              20439                       NON3DS   
                              7675                        NON3DS   

                                                                      bank_name  \
subscription_id subsegment_id                                                     
10363932201     13865645800   27718  Foreningen af Danske Kortudstedere F.M.B.A   
                              20439  Foreningen af Danske Kortudstedere F.M.B.A   
                              7675   Foreningen af Danske Kortudstedere F.M.B.A   

                                    billing_country     bin card_brand  \
subscription_id subsegment_id                                            
10363932201     13865645800   27718              DK  457105       Visa   
                              20439              DK  457105       Visa   
                              7675               DK  457105       Visa   

                                    card_category card_class  \
subscription_id subsegment_id                                  
10363932201     13865645800   27718  Visa Classic   Consumer   
                              20439  Visa Classic   Consumer   
                              7675   Visa Classic   Consumer   

                                                                 card_usage  \
subscription_id subsegment_id                                                 
10363932201     13865645800   27718  NON USA Consumer Debit - No Pin Access   
                              20439  NON USA Consumer Debit - No Pin Access   
                              7675   NON USA Consumer Debit - No Pin Access   

                                    cc_expiration_date               cid  \
subscription_id subsegment_id                                              
10363932201     13865645800   27718               0120  RETRY_DECLINED.2   
                              20439               0120  RETRY_DECLINED.2   
                              7675                0120  RETRY_DECLINED.2   

                                    date_increment  day_of_month day_of_week  \
subscription_id subsegment_id                                                  
10363932201     13865645800   27718                           25   Wednesday   
                              20439                           25   Wednesday   
                              7675                            25   Wednesday   

                                                               event_uuid  \
subscription_id subsegment_id                                               
10363932201     13865645800   27718  7159bc5a-7e30-4ce3-9183-957c7c93c66e   
                              20439  7159bc5a-7e30-4ce3-9183-957c7c93c66e   
                              7675   7159bc5a-7e30-4ce3-9183-957c7c93c66e   

                                    exp_setting_id funding_source  \
subscription_id subsegment_id                                       
10363932201     13865645800   27718                         Debit   
                              20439              

In [76]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all_07[pdf_all_07['renew_att_num'] == '3']

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
13,2018-07-25,avgstore,10363932201,7109736923,2018-07-25 06:28:30,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,25,Wednesday,7159bc5a-7e30-4ce3-9183-957c7c93c66e,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.258375,DKK,Visa,netgiro-bms,3,549,86.258375,DKK,50252786700,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-25 01:28:30,01,Authorize,0.0
14,2018-07-25,adbehkr,11831972601,7110130663,2018-07-25 07:27:32,,NON3DS,"BC Card Company, Limited",KR,490603,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0122,,,25,Wednesday,7021a0e1-8897-4055-9cb6-be6ed666984a,,Credit,KR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Declined,DR_KOREA-ENTITY,35200,31.195912,KRW,Visa,netgiro-seb,3,35200,31.195912,KRW,50257014200,27001,Transaction refused[04] [000000] [] [],New,16460118700,2018-07-25 02:27:32,02,Authorize,0.0
26,2018-07-25,kasperus,1411471609,7112583913,2018-07-25 15:47:26,,9C4C143AB183CA59,"CHASE BANK USA, N.A.",US,558250,MasterCard,MasterCard BusinessCard Card,Business,True credit (No PIN/Signature capability),0918,,,25,Wednesday,f3db86f1-46d2-4a21-8217-88360959add8,,Credit,US,DRI*Kaspersky Lab,941000108061_00000004,C5_INC-ENTITY,Declined,DR_INC-ENTITY,75.06,75.06,USD,MasterCard,mes,3,75.06,75.06,USD,50278788200,530,005: Do not honor,New,13791788100,2018-07-25 10:47:26,10,Authorize,0.0
47,2018-07-25,kasperus,3457404101,7112790993,2018-07-25 16:15:34,5,NON3DS,CAPITAL ONE BANK (CANADA BRANCH),CA,545756,MasterCard,Gold MasterCard Card,Consumer,True credit (No PIN/Signature capability),1221,,5,25,Wednesday,28e5f1fb-4599-4af8-b30f-d5a66d987df2,3809181,Credit,CA,DRI*Kaspersky Lab,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,159.99,159.99,USD,MasterCard,netgiro-bms,3,159.99,159.99,USD,50278960500,27051,Correct card number and re-try.[14] [] [144840...,New,5435866700,2018-07-25 11:15:34,11,Authorize,0.0
55,2018-07-25,kasperus,9140422001,7110738973,2018-07-25 12:17:05,,8F58D70987CD0CBC,"Chase Bank USA, National Association",US,414720,Visa,Visa Traditional,Consumer,True credit (No PIN/Signature capability),1118,RETRY_DECLINED.2,,25,Wednesday,e91f63d9-e324-4259-bb11-2005b014ce81,,Credit,US,DRI*Kaspersky Lab,7236114,DR_INC-ENTITY,Declined,DR_INC-ENTITY,99.99,99.99,USD,Visa,litle,3,99.99,99.99,USD,50275086000,530,349: Do Not Honor,New,11815740300,2018-07-25 07:17:05,07,Authorize,0.0
67,2018-07-25,avgstore,11950564101,7110236693,2018-07-25 07:38:02,0,9CBB27BD17EF7385,La Federation des Caisses Desjardins du Quebec,CA,454031,Visa,Visa Gold,Consumer,True credit (No PIN/Signature capability),1215,,0,25,Wednesday,e1b4eabc-4d2a-492f-9393-3341f87e1566,3916881,Credit,CA,DRI*AVG Technologies,7319737,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,106.99,81.363278,CAD,Visa,litle,3,106.99,81.363278,CAD,50257792400,253,327: Cardholder transaction not permitted,New,16704453100,2018-07-25 02:38:02,02,Authorize,0.0
77,2018-07-25,adbehme,11836556901,7110127613,2018-07-25 07:26:53,,NON3DS,Al Rajhi Banking and Investment Corp.,SA,405433,Visa,Visa Platinum,Consumer,NON USA Consumer Debit - No Pin Access,0520,RETRY_DECLINED.2,,25,Wednesday,6437dae2-6d7f-4c55-aff6-5b896bd8e629,,Prepaid,SA,DRI*Adobe Systems,1411

In [66]:
pdf_all[pdf_all['subscription_id'] == '1714323609'].sort_values(by=['transaction_date_in_string'])

NameError: name 'pdf_all' is not defined

In [13]:
pdf_all_07['success'] = pdf_all_07['new_status'].map({'Completed':1,'Declined':0, 'Failed':0, 'Reversed':0})

In [10]:
FEATURES_FLOAT = ['bin', 'renew_att_num', 'failed_response_code']
def handle_feat_float(df):
        for feat in FEATURES_FLOAT:
            if feat in FEATURES_FLOAT:
                df[feat] = df[feat].apply(str).str.replace('.0', '', regex=False)
        return df

handle_feat_float(pdf_all_07)    

KeyError: 'failed_response_code'

In [40]:
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    
    df_failed_txn = group[group['event_uuid'] != df_last_attempt['event_uuid']]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
#     last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
#     last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last

In [36]:
pdf_all_07['success'] = pdf_all_07['new_status'].map({'Completed':1,'Declined':0, 'Failed':0, 'Reversed':0})

In [65]:
pdf_all_07[pdf_all_07['subscription_id'] == '1714323609'].groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)

,,,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
subscription_id,subsegment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1714323609,13715900300,25450,2018-07-27,kasperde,1714323609,7121207423,2018-07-27 12:52:37,,NON3DS,VISECA CARD SERVICES S.A.,CH,540448,MasterCard,MasterCard World Card,Consumer,True credit (No PIN/Signature capability),0120,,,27,Friday,f544aa04-2617-44c4-bea9-7f81beade6b6,,Credit,CH,DRI*Kaspersky Lab,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,39.95,46.80542,EUR,MasterCard,netgiro-bms,4,39.95,46.80542,EUR,50340338000,27050,Do not try again/use alternate payment card.[0...,New,13715900300,2018-07-27 07:52:37,07,Authorize,0.0,2018-07-13 05:37:32,13,27050,Do not try again/use alternate payment card.[0...,,0,b58d134d-cb95-460d-8dca-df9f2073dbb1


In [64]:
pdf_all[pdf_all['subscription_id'] == '1852607909'].groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)

NameError: name 'pdf_all' is not defined

In [18]:
from IPython.display import display
pd.options.display.max_columns = None
display(pdf_all)
pdf_all.head()

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,fail_date_newer
0,0,2018-10-19,adbehap,11731024301,7568389263,2018-10-19 06:18:39,NaN,NON3DS,Banco del Estado de Chile,CL,456460.0,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capabil...,0921,RETRY_DECLINED.2,NaN,19,friday,4eb8e5c1-5bd3-4562-a065-b75506e65376,NaN,Credit,CL,DRI*Adobe Sales,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,13600.00,20.300000,CLP,Visa,netgiro-bms,6.0,13600.00,20.300000,CLP,53268075000,27001,Transaction refused[05] [] [111800] [111800],New,1.707147e+10,2018-10-19 01:18:39,1,Authorize,0,2018-09-19 01:57:47,19,27083,Insufficient Funds[51] [] [111800] [111800],Insufficient Funds,30,afab242a-2436-4896-9f7e-a136d0664bd5,27.588374,4.468553,675.693909,24.260824,56.487480,69.990000,10.307784,False
1,1,2018-05-20,tmapac,10083022901,6803738323,2018-05-20 12:00:07,NaN,NON3DS,IMB Limited,AU,456441.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1218,RETRY_DECLINED.2,NaN,5,sunday,e04f02b6-4644-403b-b82f-02fe393af83b,NaN,Debit,AU,DRI*Trend Micro,1678684472,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,39.95,30.029410,AUD,Visa,drwp-fd,3.0,39.95,30.029410,AUD,48120171700,27050,Do not try again/use alternate payment card[nu...,New,1.339742e+10,2018-05-20 12:00:07,12,Authorize,0,2018-05-13 01:09:47,5,521,51: Insufficient funds,Insufficient Funds,7,18273093-db83-4ea1-b2cc-7f474b101f8d,59.674779,2.160000,568.621749,47.217585,111.383369,138.033361,21.777957,False
2,2,2018-07-14,adbehbr,9903708401,7055340463,2018-07-14 06:24:46,NaN,F05553311D75553D,Banco do Brasil S.A.,BR,498401.0,Visa,Visa Platinum,Consumer,True credit (No PIN/Signature capability),0720,NaN,NaN,14,saturday,949f9108-8da6-484e-8d6a-edff342d1b84,NaN,Credit,BR,DRI*Adobe Systems,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Completed,DR_BRAZIL-ENTITY,35.00,8.981222,BRL,Visa,pagador,4.0,35.00,8.981222,BRL,49906581900,100,0:Successful,New,1.629412e+10,2018-07-14 01:24:46,1,Authorize,1,2018-07-07 01:19:02,7,530,7: Do Not Honor,do_not_honor,7,845af543-7978-4961-abf6-47543aacb026,21.383127,2.490000,566.000000,16.589026,47.986260,61.030274,9.030781,False
3,3,2018-04-02,avast,9938359401,6625321463,2018-04-02 01:01:45,3.0,NON3DS,Banque Malgache de L'Ocean Indien,MG,423365.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1220,RETRY_DECLINED.2,NaN,2,monday,f4f3bcbf-e09f-4ec7-817f-8755f1122c76,NaN,Prepaid,MG,DRI*AVAST Software,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,59.99,59.990000,USD,Visa,netgiro-seb,3.0,59.99,59.990000,USD,46458403400,27001,Transaction refused[05] [000000] [] [],New,1.316008e+10,2018-04-02 01:01:45,1,Authorize,0,2018-03-18 01:06:44,18,27001,Transaction refused[05] [] [111800] [111800],txn_refused,15,be5ab061-01e9-4a78-8990-ce8365d81001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,2018-07-27,kasperde,1714323609,7121207423,2018-07-27 12:52:37,NaN,NON3DS,VISECA CARD SERVICES S.A.,CH,540448.0,MasterCard,MasterCard World Card,Consumer,True credit (No PIN/Signature capability),0120,NaN,NaN,27,friday,f544aa04-2617-44c4-bea9-7f81beade6b6,NaN,Credit,CH,DRI*Kaspersky Lab,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,39.95,46.80542

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,fail_date_newer
0,0,2018-10-19,adbehap,11731024301,7568389263,2018-10-19 06:18:39,NaN,NON3DS,Banco del Estado de Chile,CL,456460.0,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capabil...,0921,RETRY_DECLINED.2,NaN,19,friday,4eb8e5c1-5bd3-4562-a065-b75506e65376,NaN,Credit,CL,DRI*Adobe Sales,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,13600.00,20.300000,CLP,Visa,netgiro-bms,6.0,13600.00,20.300000,CLP,53268075000,27001,Transaction refused[05] [] [111800] [111800],New,1.707147e+10,2018-10-19 01:18:39,1,Authorize,0,2018-09-19 01:57:47,19,27083,Insufficient Funds[51] [] [111800] [111800],Insufficient Funds,30,afab242a-2436-4896-9f7e-a136d0664bd5,27.588374,4.468553,675.693909,24.260824,56.487480,69.990000,10.307784,False
1,1,2018-05-20,tmapac,10083022901,6803738323,2018-05-20 12:00:07,NaN,NON3DS,IMB Limited,AU,456441.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1218,RETRY_DECLINED.2,NaN,5,sunday,e04f02b6-4644-403b-b82f-02fe393af83b,NaN,Debit,AU,DRI*Trend Micro,1678684472,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,39.95,30.029410,AUD,Visa,drwp-fd,3.0,39.95,30.029410,AUD,48120171700,27050,Do not try again/use alternate payment card[nu...,New,1.339742e+10,2018-05-20 12:00:07,12,Authorize,0,2018-05-13 01:09:47,5,521,51: Insufficient funds,Insufficient Funds,7,18273093-db83-4ea1-b2cc-7f474b101f8d,59.674779,2.160000,568.621749,47.217585,111.383369,138.033361,21.777957,False
2,2,2018-07-14,adbehbr,9903708401,7055340463,2018-07-14 06:24:46,NaN,F05553311D75553D,Banco do Brasil S.A.,BR,498401.0,Visa,Visa Platinum,Consumer,True credit (No PIN/Signature capability),0720,NaN,NaN,14,saturday,949f9108-8da6-484e-8d6a-edff342d1b84,NaN,Credit,BR,DRI*Adobe Systems,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Completed,DR_BRAZIL-ENTITY,35.00,8.981222,BRL,Visa,pagador,4.0,35.00,8.981222,BRL,49906581900,100,0:Successful,New,1.629412e+10,2018-07-14 01:24:46,1,Authorize,1,2018-07-07 01:19:02,7,530,7: Do Not Honor,do_not_honor,7,845af543-7978-4961-abf6-47543aacb026,21.383127,2.490000,566.000000,16.589026,47.986260,61.030274,9.030781,False
3,3,2018-04-02,avast,9938359401,6625321463,2018-04-02 01:01:45,3.0,NON3DS,Banque Malgache de L'Ocean Indien,MG,423365.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1220,RETRY_DECLINED.2,NaN,2,monday,f4f3bcbf-e09f-4ec7-817f-8755f1122c76,NaN,Prepaid,MG,DRI*AVAST Software,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,59.99,59.990000,USD,Visa,netgiro-seb,3.0,59.99,59.990000,USD,46458403400,27001,Transaction refused[05] [000000] [] [],New,1.316008e+10,2018-04-02 01:01:45,1,Authorize,0,2018-03-18 01:06:44,18,27001,Transaction refused[05] [] [111800] [111800],txn_refused,15,be5ab061-01e9-4a78-8990-ce8365d81001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,2018-07-27,kasperde,1714323609,7121207423,2018-07-27 12:52:37,NaN,NON3DS,VISECA CARD SERVICES S.A.,CH,540448.0,MasterCard,MasterCard World Card,Consumer,True credit (No PIN/Signature capability),0120,NaN,NaN,27,friday,f544aa04-2617-44c4-bea9-7f81beade6b6,NaN,Credit,CH,DRI*Kaspersky Lab,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,39.95,46.80542

In [86]:
bin_profile = pd.read_csv(WORK_DIR + 'bin_profile.csv')
bin_profile.shape



(30600, 10)

In [87]:
bin_profile['bin'] = bin_profile['bin'].apply(str).str.replace('.0', '', regex=False)
bin_profile.head()

,Unnamed: 0,bin,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,count
0,0,nan,10.231983,0.011639,410.000000,19.355269,57.480000,85.590000,4.590894,3026
1,5,352818,24.845652,4.839536,51.479167,19.995024,47.495588,50.682451,4.840481,5
2,7,352821,49.015144,48.102689,50.518630,0.988569,50.246824,50.464269,48.115695,5
3,8,352822,33.641090,18.714051,49.360468,10.450449,46.485601,48.785495,18.714051,9
4,12,352831,39.761784,18.336803,68.206773,17.110155,62.866110,67.138640,18.336803,8


In [90]:
pdf_all = pd.merge(pdf_all, bin_profile[['bin','Mean','Min','Max','StdDev','Max_95','Max_99','Min_1']], left_on='bin', right_on='bin', how='left')
pdf_all.shape

(5589753, 60)

In [91]:
pdf_all.to_csv(WORK_DIR + 'retry_success_last_two_with_bin.csv')

In [94]:
pdf_all.head(10)

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1
0,2018-08-16,avast,10464089601,7217760623,2018-08-16 08:45:15,,NON3DS,UBS SWITZERLAND AG,CH,540659,MasterCard,Prepaid Unembossed MasterCard Card (Non-US),Consumer,True credit (No PIN/Signature capability),0820,RETRY_DECLINED.2,,16,thursday,ae08384e-5ec6-49bd-9c84-bf22c7766b70,,Credit,CH,DRI*AVAST Software,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,59.9,60.069794,CHF,MasterCard,netgiro-seb,2,59.9,60.069794,CHF,51002708000,27001,Transaction refused[05] [000000] [] [],New,14048665100,2018-08-16 03:45:15,03,Authorize,0,2018-08-16 03:45:14,16,27001,Transaction refused[05] [] [111800] [111800],txn_refused,0,a418bb21-523a-4057-b426-926dabea8e5b,47.871312,2.680000,214.067655,28.253103,91.398280,112.773363,8.694314
1,2018-06-15,avast,9593217601,6909821493,2018-06-15 05:49:58,,NON3DS,La Federation des Caisses Desjardins du Quebec,CA,454033,Visa,Visa Gold,Consumer,True credit (No PIN/Signature capability),0920,RETRY_DECLINED.2,,15,friday,f8384b75-34ad-4d5c-986d-6703be9e1cbd,,Credit,CA,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,59.99,46.228006,CAD,Visa,netgiro-bms,3,59.99,46.228006,CAD,48965323200,27001,Transaction refused[05] [] [124800] [124800],New,12533800900,2018-06-15 00:49:58,00,Authorize,0,2018-06-15 00:49:58,15,530,349: Do Not Honor,do_not_honor,0,910e4438-b77d-490c-91bf-e3668688027f,59.876459,2.490000,928.114155,32.373010,109.990000,159.990000,19.824968
2,2018-10-14,avgstore,10641108501,7541814153,2018-10-14 15:45:45,0,775ED0F646DEAADF,STAR PROCESSING INC.,US,551117,MasterCard,Standard Debit MasterCard,Consumer,Debit - Debit Hybrid (PIN and Signature),0818,RETRY_DECLINED.2,0,14,sunday,f7960229-3523-4e85-9b6f-a48a4fa35df4,406483,Debit,US,DRI*AVG Technologies,7236114,DR_INC-ENTITY,Declined,DR_INC-ENTITY,52.99,52.99,USD,MasterCard,litle,7,52.99,52.99,USD,53113310100,303,350: Generic Decline,New,14368496400,2018-10-14 10:45:45,10,Authorize,0,2018-10-14 10:45:44,14,530,005: Do not honor,do_not_honor,0,b5aba462-0ff9-4ec7-b5f7-1e8eb022a3a3,59.880204,8.470000,105.990000,24.577707,88.378000,100.882800,8.470000
3,2018-08-22,avast,9218957201,7248181183,2018-08-22 06:17:32,3,NON3DS,MASTER RED EUROPA S.L.,ES,558436,MasterCard,MasterCard BusinessCard Card,Business,True credit (No PIN/Signature capability),1120,RETRY_DECLINED.2,,22,wednesday,e2cf9b19-3caf-49eb-a932-23a2f79240ad,,Credit,ES,DRI*AVAST Software,1009072982,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,59.99,69.000498,EUR,MasterCard,drwp-fd,3,59.99,69.000498,EUR,51203602700,27001,Transaction refused[null] [null] [null] [null],New,11939715300,2018-08-22 01:17:32,01,Authorize,0,2018-08-22 01:17:30,22,27001,Transaction refused[05] [] [111800] [111800],txn_refused,0,99b53f1e-7f43-44a8-8036-10be2a2047f6,66.389442,0.980000,196.585554,39.373117,137.565628,195.119470,7.152872
4,2018-08-20,avgstore,12006605701,7238248363,2018-08-20 07:00:46,3,180820,BANCO BPM SOCIETA PER AZIONI,IT,540033,MasterCard,Prepaid Unembossed MasterCard Card (Non-US),Consumer,True credit (No PIN/Signature capability),0320,RETRY_DECLINED.2,,20,monday,140efcce-089d-4dc7-84c2-2710d096f808,,Credit,IT,AVG*AVG Technologies,207686,AVG-ENTITY,Declined,DR_I

In [93]:
sc.stop()

In [2]:
import pandas as pd
WORK_DIR = '/var/spark/ml_files/'
pdf_all = pd.read_csv(WORK_DIR + 'retry_success_with_bin_01_to_10.csv')
pdf_all.shape

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (39,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(10761324, 61)

In [15]:
import time
import datetime
from datetime import timedelta
#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

def fail_date_newer(row):
    try:
        fail_date = to_date(row['failed_attempt_date'])
        txn_date = to_date(row['transaction_date_in_string'])
    except Exception as ex:
        print("Found exception converting date for row: {}. Ex:{}".format(row, ex))
        return True
    return fail_date > txn_date

pdf_all['fail_date_newer'] = pdf_all.apply(fail_date_newer, axis=1)

In [32]:
pdf_all['fail_date_newer'].value_counts()

False    10758721
True         2603
Name: fail_date_newer, dtype: int64

In [29]:
pdf_all_07[pdf_all_07['subscription_id'] == '11623213001'].sort_values(by=['transaction_date'])   #.apply(fail_date_newer, axis=1)

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type
128980,2018-07-05,adbeheu,11623213001,7010995093,2018-07-05 08:58:00,,NON3DS,Raiffeisen Bank Aval PJSC,UA,414950,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0519,,,5,Thursday,88304f5c-5950-4c3b-918d-c6adacb3e826,,Debit,UA,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,19.99,19.99,USD,Visa,netgiro-bms,2,19.99,19.99,USD,49630854500,27083,Insufficient Funds[51] [] [111800] [111800],New,16344560000,2018-07-05 03:58:00,03,Authorize
38561,2018-07-12,adbeheu,11623213001,7045130773,2018-07-12 09:29:16,,NON3DS,Raiffeisen Bank Aval PJSC,UA,414950,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0519,,,12,Thursday,c13558dc-7620-4278-a9ea-4eee8b26bafd,,Debit,UA,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,19.99,19.99,USD,Visa,netgiro-bms,3,19.99,19.99,USD,49851923000,0,SUCCESS[00] [680757] [111800] [111800],New,16344560000,2018-07-12 04:29:16,04,Authorize
84357,2018-07-19,adbeheu,11623213001,7080290213,2018-07-19 07:34:01,,NON3DS,Raiffeisen Bank Aval PJSC,UA,414950,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0519,,,19,Thursday,3019593d-e0fd-4778-b708-e8167a424546,,Debit,UA,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,19.99,19.99,USD,Visa,netgiro-bms,4,19.99,19.99,USD,50068133100,27083,Insufficient Funds[51] [] [111800] [111800],New,16344560000,2018-07-19 02:34:01,02,Authorize
49973,2018-07-26,adbeheu,11623213001,7115287593,2018-07-26 08:35:55,,NON3DS,Raiffeisen Bank Aval PJSC,UA,414950,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0519,,,26,Thursday,a34194cc-c908-4c7d-923d-e45f63e63965,,Debit,UA,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,19.99,19.99,USD,Visa,netgiro-bms,5,19.99,19.99,USD,50288838800,27083,Insufficient Funds[51] [] [111800] [111800],New,16344560000,2018-07-26 03:35:55,03,Authorize
58426,2018-07-28,adbeheu,11623213001,7126214343,2018-07-28 08:53:04,,NON3DS,Raiffeisen Bank Aval PJSC,UA,414950,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0519,,,28,Saturday,ebe33a29-5c19-4972-a5eb-d8fd9f7a7285,,Debit,UA,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,19.99,19.99,USD,Visa,netgiro-bms,6,19.99,19.99,USD,50360062900,27083,Insufficient Funds[51] [] [111800] [111800],New,16344560000,2018-07-28 03:53:04,03,Authorize


In [16]:
pdf_all[pdf_all['fail_date_newer'] == True][['subscription_id', 'transaction_date_in_string', 'failed_attempt_date']]

,subscription_id,transaction_date_in_string,failed_attempt_date
1150,11864530001,2018-07-18 00:33:10,2018-08-01 00:32:20
2770,10217440101,2018-07-18 01:20:12,2018-07-25 01:18:07
5456,10129306301,2018-06-24 01:27:13,2018-07-15 01:20:35
5609,10129306301,2018-06-24 01:27:13,2018-07-08 01:22:16
9545,10361867901,2018-02-25 01:14:02,2018-03-04 01:30:40
10182,10873546601,2018-03-30 01:10:17,2018-04-01 01:27:40
14621,3672757701,2018-10-06 00:35:14,2018-10-16 00:31:46
17033,10870952701,2018-06-28 03:29:18,2018-07-05 03:11:54
21032,3995102101,2018-05-26 00:40:00,2018-06-23 00:54:27
22015,10100756301,2018-01-17 01:27:52,2018-01-18 01:14:07


In [22]:
# pdf_all.to_csv(WORK_DIR + 'retry_success_with_bin_01_to_10.csv')

In [6]:
# from sklearn.preprocessing import Imputer
# # import sklearn.preprocessing.Imputer as imputer
# sklearn.preprocessing.__version__
import sklearn
sklearn.__version__

'0.19.2'

In [8]:
import pandas as pd
WORK_DIR = '/var/spark/ml_files/'
bin_profile =  pd. read_csv(WORK_DIR + 'bin_profile.csv')

In [9]:
bin_profile.head()

,Unnamed: 0,bin,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,count
0,4,352818.0,24.845652,4.839536,51.479167,19.995024,47.495588,50.682451,4.840481,5
1,6,352821.0,49.015144,48.102689,50.518630,0.988569,50.246824,50.464269,48.115695,5
2,7,352822.0,33.641090,18.714051,49.360468,10.450449,46.485601,48.785495,18.714051,9
3,11,352831.0,39.761784,18.336803,68.206773,17.110155,62.866110,67.138640,18.336803,8
4,12,352832.0,39.497431,31.606969,54.133242,9.201100,51.345292,53.575652,31.625561,5


In [14]:
bin_profile[bin_profile['bin'] == 456460.0]

,Unnamed: 0,bin,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,count
13898,19314,456460.0,27.588374,4.468553,675.693909,24.260824,56.48748,69.99,10.307784,1230
